In [ ]:
import json
from py2neo import Graph, Node, Relationship
#from py2neo.Graph import database 

# Need to get authentication working, currently NEO4J_AUTH=none
graph = Graph("bolt://neo4j:7687")
# graph = Graph('bolt://localhost:7687', bolt=True)

#graph.delete_all()

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

In [ ]:
MATCH (q:Quanta) 
WHERE EXISTS(q.fos)  AND (q.n_citation>100) AND (q.venue="Nature" OR q.venue="Science") AND (q.year>=2010)
WITH q.title as title, q.venue as venue, q.year as year, q.n_citation as n_citation, q.pageRank_2018 as pagerank,
	SIZE((q)<-[:CITES]-(:Quanta {year: q.year+1})) as c1,
    SIZE((q)<-[:CITES]-(:Quanta {year: q.year+2})) as c2,
    SIZE((q)<-[:CITES]-(:Quanta {year: q.year+3})) as c3,
    SIZE((q)<-[:CITES]-(:Quanta {year: q.year+4})) as c4,
	SIZE((q)<-[:CITES]-(:Quanta {year: q.year+5})) as c5,
	SIZE((q)<-[:CITES]-(:Quanta {year: q.year+6})) as c6
WITH *,
	CASE WHEN (c2 > 1.25*c1) THEN 1 ELSE 0 END as b1,
	CASE WHEN (c3 > 1.25*c2) THEN 1 ELSE 0 END as b2,	
 	CASE WHEN (c4 > 1.25*c3) THEN 1 ELSE 0 END as b3,
    CASE WHEN (c5 > 1.25*c4) THEN 1 ELSE 0 END as b4,
    CASE WHEN (c6 > 1.25*c5) THEN 1 ELSE 0 END as b5
WHERE 
	(b1+b2+b3+b4+b5)>=3
RETURN title, venue, year, n_citation, c1, c2, c3, c4, c5, c6, pagerank, (b1+b2+b3+b4+b5) as sum_bool